In [1]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels
from statsmodels.formula.api import ols
from scipy import stats
import math
import seaborn as sns
import numpy as np
import pandas as pd
import csv

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

In [2]:
raw_data = pd.read_csv('data/data_cleaned.csv')
raw_data.reset_index(drop=False, inplace=True)
data_df = raw_data
data_df['Salary_sqrt'] = data_df['Salary'] ** (1/2)
data_df.head()

,index,IDfg,Name,Age,G,PA,AB,R,H,1B,2B,3B,HR,RBI,BB,SO,HBP,IBB,SB,CS,GDP,SH,SF,AVG,OBP,SLG,OPS,ISO,BABIP,wRAA,wRC+,WPA,WAR,Year_FA,seasons_included,Salary,Salary_sqrt
0,0,5677,A.J. Ellis,35,468,1713,1448,143,341,241,63,2,35,172,214,311,17,20,2,3,52,19,15,0.235497,0.345391,0.354282,0.699673,0.118785,0.273948,-1.68,92.8,-1.46,2.6,2017,5,2500000.0,1581.138830
1,1,746,A.J. Pierzynski,36,664,2587,2424,254,682,479,122,7,74,300,101,278,29,21,5,6,76,12,21,0.281353,0.320878,0.429043,0.749921,0.147690,0.290492,1.20,94.6,-2.58,8.8,2014,5,500000.0,707.106781
2,2,746,A.J. Pierzynski,35,664,2628,2455,272,695,489,129,7,70,290,109,273,28,24,5,5,76,15,21,0.283096,0.324611,0.426884,0.751495,0.143788,0.293015,0.00,93.6,-2.85,9.7,2013,5,7500000.0,2738.612788
3,3,746,A.J. Pierzynski,37,628,2414,2258,222,616,431,112,7,66,288,91,280,33,17,4,6,71,10,22,0.272808,0.312681,0.416298,0.728978,0.143490,0.284385,-0.48,90.4,-4.54,5.3,2015,5,2000000.0,1414.213562
4,4,746,A.J. Pierzynski,38,613,2347,2191,217,610,429,107,8,66,281,95,278,34,17,1,4,73,4,23,0.278412,0.320339,0.424920,0.745259,0.146508,0.290909,2.42,96.6,-2.46,5.1,2016,5,3000000.0,1732.050808


In [3]:
predictors = ['Age', 'G', 'PA', 'AB', 'R', 'H', '1B',
               '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'HBP', 'IBB', 'SB', 'CS',
               'GDP', 'SH', 'SF', 'AVG', 'OBP', 'SLG', 'OPS', 'ISO', 'BABIP',
               'wRAA', 'wRC+', 'WPA', 'WAR', 'Year_FA']


response = 'Salary_sqrt'
# predictors = ['Age', 'AB', 'R', 'H', 'WAR']
alpha_enter = 0.15
alpha_remove = 0.15
alpha = 0.15
var_to_fit = []
pvalues_new = {}
pvalues_old = {}
var_selected = []
var_removed = []
var_kept = []

pvalues_final = {}

In [4]:
def stepwise(predictors, response, alpha):
    def clear_values():
        pvalues_new.clear()
        pvalues_old.clear()
        var_removed = []
        var_kept = []    
    
    def fit_model(x):
        pvalues_old.clear()
        X = data_df[x]
        y = data_df[response]
        X = sm.add_constant(X)
        model = sm.OLS(y, X)
        results = model.fit()
        for i in range(len(x)-1):
            pvalues_old[x[i]] = results.pvalues[i+1]
        pvalues_new[x[-1]] = results.pvalues[-1]
    
    def check_pvalues_new(val):
        """check if pvalues_new contains p<alpha"""
        for key in pvalues_new:
            if pvalues_new[key] <= val:
                return True
                break
            else:
                continue
        return False

    def check_pvalues_old(val):
        """check if pvalues_old contains p>alpha"""

        for key in pvalues_old:
            if pvalues_old[key] > val:
                return True
                break
            else:
                continue
        return False
    
    
    count = 1
    keep_going = True
    while keep_going:
        print(f"Round: {count}")
        print(f"Predictors: {predictors}")
        clear_values()

        for i in range(len(predictors)):
            var_to_fit = var_selected + [predictors[i]]
            fit_model(var_to_fit)
        print(f"pvalues_new: {pvalues_new}")

        if check_pvalues_new(alpha):
            var_min = min(pvalues_new, key=pvalues_new.get)
            print(f"var min: {var_min}")
            var_to_fit = var_selected + [var_min]
            fit_model(var_to_fit)
            if check_pvalues_old(alpha):
                for var in var_selected:
                    if pvalues_old[var] > alpha:
                        var_remove = var
                    else:
                        continue
                    var_selected.remove(var)
                    predictors.append(var)
                    var_removed.append(var)
                var_kept = var_selected
            else:
                var_kept = var_selected
            print(f"Var kept: {var_kept}")
            var_selected.append(var_min)
            predictors.remove(var_min)
            print(f"var removed: {var_removed}")
            print(f"var_selected: {var_selected}")
            print("------------------------------------------------------------------------")
            count += 1
        else:   
            X = data_df[var_selected]
            y = data_df[response]
            X = sm.add_constant(X)
            model = sm.OLS(y, X)
            results = model.fit()
            for i in range(len(var_selected)):
                pvalues_final[var_selected[i]] = results.pvalues[i+1]

            keep_going = False
            print(f"Stepwise completed. Vars selected: {var_selected}")


In [5]:
stepwise(predictors, response, alpha)

Round: 1
Predictors: ['Age', 'G', 'PA', 'AB', 'R', 'H', '1B', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'HBP', 'IBB', 'SB', 'CS', 'GDP', 'SH', 'SF', 'AVG', 'OBP', 'SLG', 'OPS', 'ISO', 'BABIP', 'wRAA', 'wRC+', 'WPA', 'WAR', 'Year_FA']
pvalues_new: {'Age': 0.0076052187411208725, 'G': 1.4880988363653962e-35, 'PA': 1.6441662670795335e-50, 'AB': 7.859159737962572e-48, 'R': 5.974775659752728e-67, 'H': 6.223903169768053e-54, '1B': 2.8734652230319394e-35, '2B': 2.4317060032343706e-54, '3B': 1.1068771339420233e-11, 'HR': 1.5104638370606087e-53, 'RBI': 1.4839839946512421e-58, 'BB': 4.4859554265937694e-42, 'SO': 2.7170679509970476e-31, 'HBP': 9.43268159485875e-10, 'IBB': 1.2768711618774809e-21, 'SB': 1.28982100090231e-07, 'CS': 1.8228052681862362e-08, 'GDP': 2.0412577389992458e-23, 'SH': 0.00016037157461732527, 'SF': 2.318797000745184e-23, 'AVG': 2.923845427437395e-29, 'OBP': 1.591009241519146e-37, 'SLG': 2.607325260973996e-55, 'OPS': 1.8212810940586554e-62, 'ISO': 6.1302297369617794e-30, 'BABIP': 2.4

pvalues_new: {'G': 0.12749773330933697, 'PA': 0.10208535660988771, 'AB': 0.06617739952363696, 'R': 0.2693908821097478, 'H': 0.04947852762898053, '1B': 0.09729744401011593, '2B': 0.039584874345178614, '3B': 0.5565902899873447, 'HR': 0.031634899271418326, 'RBI': 0.02402776172272079, 'BB': 0.9953454956223705, 'SO': 0.3358795591373107, 'HBP': 0.13057943681786752, 'IBB': 0.5219575048803988, 'SB': 0.9505298442004051, 'CS': 0.9034686705756592, 'GDP': 0.23445936600763043, 'SH': 0.2539028154152657, 'SF': 0.3587871025171412, 'OBP': 0.011716901820052902, 'OPS': 0.011716901820051988, 'BABIP': 0.065419366376765, 'wRC+': 0.3573712193709325, 'WPA': 0.22091127142118996, 'ISO': 0.13889655120842126}
var min: OPS
Var kept: ['WAR', 'SLG', 'Age', 'Year_FA', 'AVG', 'wRAA']
var removed: ['ISO']
var_selected: ['WAR', 'SLG', 'Age', 'Year_FA', 'AVG', 'wRAA', 'OPS']
------------------------------------------------------------------------
Round: 9
Predictors: ['G', 'PA', 'AB', 'R', 'H', '1B', '2B', '3B', 'HR', 'R

In [7]:
X = data_df[var_selected]
y = data_df[response]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            Salary_sqrt   R-squared:                       0.662
Model:                            OLS   Adj. R-squared:                  0.656
Method:                 Least Squares   F-statistic:                     107.6
Date:                Wed, 02 Feb 2022   Prob (F-statistic):          1.60e-122
Time:                        12:34:04   Log-Likelihood:                -4420.7
No. Observations:                 561   AIC:                             8863.
Df Residuals:                     550   BIC:                             8911.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -9.037e+04   1.95e+04     -4.624      0.0

In [9]:
results.aic

8863.40834981039